In [1]:
from sec_edgar_api import EdgarClient
import json
import os

In [32]:
def get_filings(ciks, target_year):
    edgar = EdgarClient(user_agent="<Sample Company Name> <Admin Contact>@<Sample Company Domain>")

    # List to collect all 10-K filings for the target year across companies
    all_filings_for_year = []

    # Loop through each CIK (company)
    for cik in ciks:
        print(f"Getting 10-K filings for CIK: {cik}...")
        data = edgar.get_submissions(cik)
        
        # Extract the company name (tickers) from the submission data
        company = data.get('tickers', [])[0] if 'tickers' in data else "Unknown_Company"
        
        recent_filings = data.get('filings', {}).get('recent', {})
        forms = recent_filings.get('form', [])
        accession_numbers = recent_filings.get('accessionNumber', [])
        filing_dates = recent_filings.get('filingDate', [])

        # Filter for 10-K filings from the specific year
        ten_k_filings = [
            {"company": company, "accessionNumber": accession, "filingDate": filing_date}
            for form, accession, filing_date in zip(forms, accession_numbers, filing_dates)
            if form == "10-K" and filing_date.startswith(str(target_year))
        ]

        # Add the filtered 10-K filings to the all_filings_for_year list
        all_filings_for_year.extend(ten_k_filings)

        # Prepare formatted URLs for each 10-K filing
        for filing in ten_k_filings:
            accession_number = filing.get("accessionNumber", "")
            filing_date = filing.get("filingDate", "")
            company = filing.get("company", "")
            
            if accession_number and filing_date:
                year = filing_date[:4]  # Extract the year from the filing date
                formatted_accession = accession_number.replace("-", "")
                url = f"https://www.sec.gov/Archives/edgar/data/{cik}/{formatted_accession}/{accession_number}.txt"
                filing["url"] = url  # Add the URL to the filing

    # Save all 10-K filings for the target year in a single JSON file
    output_file_path = f'./10K-URL/All_10K_Filings_{target_year}.json'
    os.makedirs(os.path.dirname(output_file_path), exist_ok=True)  # Create directory if it doesn't exist
    
    # Write the accumulated filings to a JSON file
    with open(output_file_path, 'w') as output_file:
        json.dump(all_filings_for_year, output_file, indent=4)

In [34]:
# AAPL, AMZN, NVDA, NFLX, IBM, QUALCOMM, TWITTER
ciks = ["0000320193", "0001018724", "0001045810", "0001065280", "0001318605", "0000789019"]  # Example CIKs for Apple, Microsoft, and Google
target_year = 2020
get_filings(ciks, target_year)


Getting 10-K filings for CIK: 0000320193...
Getting 10-K filings for CIK: 0001018724...
Getting 10-K filings for CIK: 0001045810...
Getting 10-K filings for CIK: 0001065280...
Getting 10-K filings for CIK: 0001318605...
Getting 10-K filings for CIK: 0000789019...
